# Training Neural Networks

이전 파트에서 구성했던 network는 똑똑하지 않다. 왜냐하면 숫자 솔글씨에 대해서 아는 것이 없었다. 비선형 activations를 가지는 Neural network은 universal function approximators처럼 동작한다. 여기에는 input을 output으로 매핑하는 함수가 있다. 예제로 숫자 손글씨의 images를 class 확률로 매핑한다. neural networks의 파워는 훈련시켜서 이 함수가 근사치에 맞게 하고 충분히 주어진 데이터 함수와 계산 시간을 근사치에 맞게 한다.

<img src="assets/function_approx.png" width=500px>

먼저 network의 초기에는 inputs을 output으로 매핑하는 함수에 대해서 아는 것이 없다. 실제 데이터의 예제를 보여서 network를 train시키고 다음으로 network parameters를 조정해서 이 함수를 approximates 시킨다.

이런 parameters를 찾기 위해서 network가 실제 output과 추정치가 얼마나 다른지를 알아야한다. 이를 위해서 **loss function** (cost 라고 부르는) 를 계산으로 prediction error의 측정한다. 예제로 mean squared loss는 regression과 binary classification 문제에 사용된다.

$$
\large \ell = \frac{1}{2n}\sum_i^n{\left(y_i - \hat{y}_i\right)^2}
$$

$n$ 는 training 예제의 개수이고 $y_i$ 는 true labels의 개수, $\hat{y}_i$ 는 추정 labels의 개수이다.

network parameters에 대한 이 loss를 최소화시켜서, loss가 최소이고 높은 정확도를 가지고 제대로 labels를 추정할 수 있는 configurations를 찾을 수 있다. **gradient descent** 이라는 프로세스를 사용해서 이 최소값을 찾는다. gradient는 loss function의 slope이고 가장 빠르게 변화되는 방향에 있는 지점이다. 최소 시간에 이 최소값을 얻기 위해서 gradient(downwards)를 따른다. base에 대해서 가장 급경사인 곳을 따라서 산을 내려가는 것을 생각하면 된다.

<img src='assets/gradient_descent.png' width=350px>

## Backpropagation

single layer networks에 대해서 gradient descent는 구현이 간단하다. 하지만 우리가 만든 것과 같이 더 deep하고 multilayer neural networks에 대해서는 더 복잡한다. 연구원이 multilayer networks를 train시키는 것을 이해하려면 30년은 족히 걸린다.

multilayer networks를 train시키는 것은 **backpropagation** 을 통해서 행해진다. 이는 실제로 미적분에서 chain rule을 단순하게 적용한 것이다. 2 layer network을 graph 표현으로 변환한다고 이해하는게 가장 쉬울 것이다.

<img src='assets/backprop_diagram.png' width=550px>

network를 통한 전방 통과에서 data와 operation은 바닥에서 꼭대기로 간다. input $x$ 를 linear transformation $L_1$ 에 weights $W_1$ 와 biases $b_1$ 으로 통과시킨다. 다음으로 output은 sigmoid operation $S$ 와 또 다른 linear transformation $L_2$ 를 수행한다. 마지막으로 loss $\ell$ 를 계산한다. network의 prediction이 얼마나 안좋은지 측정으로 loss를 사용한다. 다음 목표는 weights와 biases를 조정하여 loss를 최소화시키는 것이다.

weights를 gradient descent와 train하기 위해서 loss backwards의 gradient를 network로 전파시킨다. 각 operation은 input과 output 사이에 gradient를 가진다. gradient backwards를 보냄에 따라 incoming gradient를 gradient로 곱한다. 수학적으로 이것은 실제로는 단지 chain rule을 사용해서 weights에 대한 loss의 gradient를 계산하는 것이다.

$$
\large \frac{\partial \ell}{\partial W_1} = \frac{\partial L_1}{\partial W_1} \frac{\partial S}{\partial L_1} \frac{\partial L_2}{\partial S} \frac{\partial \ell}{\partial L_2}
$$

**Note:** 여기서 몇가지를 설명 안하고 그냥 넘어갔는데 vector 미적분학에 대한 지식이 필요하지만 무엇을 하고 있는지를 이해하는데는 상관이 없어서 넘어갔다.

learning rate $\alpha$를 가지는 이런 gradient를 사용해서 weights를 업데이트하자.

$$
\large W^\prime_1 = W_1 - \alpha \frac{\partial \ell}{\partial W_1}
$$

learning rate $\alpha$ 는 iterative method가 최소가 되도록 weight update step은 충분히 작게 설정한다.

## Import Resources

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', tf.keras.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

## Load the Dataset

In [ ]:
training_set, dataset_info = tfds.load('mnist', split='train', as_supervised = True, with_info = True)

## Create Pipeline

In [ ]:
def normalize(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

num_training_examples = dataset_info.splits['train'].num_examples

batch_size = 64

training_batches = training_set.cache().shuffle(num_training_examples//4).batch(batch_size).map(normalize).prefetch(1)

## Build the Model

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(10, activation = 'softmax')
])

## Getting the Model Ready For Training

우리가 만든 model을 train시키기 전에 parameters를 설정해서 train 키는데 사용한다. `.compile` method를 사용해서 train을 위해서 model을 configure할 수 있다. `.compile` method에서 지정이 필요한 main parameters들은 다음과 같다 :

* **Optimizer:** training하는 동안 model의 weight를 업데이트하는데 사용하는 알고리즘이다. 이 레슨을 통해서 [`adam`](http://arxiv.org/abs/1412.6980) optimizer를 사용할 예정이다. Adam은 stochastic gradient descent 알고리즘이다. `tf.keras` 에서 유효한 optimizers에 대한 전체 목록은 [optimizers documentation](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/optimizers#classes)를 확인하자.


* **Loss Function:** The loss function we are going to use during training to measure the difference between the true labels of the images in your dataset and the predictions made by your model. In this lesson we will use the `sparse_categorical_crossentropy` loss function. We use the `sparse_categorical_crossentropy` loss function when our dataset has labels that are integers, and the `categorical_crossentropy` loss function when our dataset has one-hot encoded labels. For a full list of the loss functions available in `tf.keras` check out the [losses documentation](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/losses#classes).


* **Metrics:** A list of metrics to be evaluated by the model during training. Throughout these lessons we will measure the `accuracy` of our model. The `accuracy` calculates how often our model's predictions match the true labels of the images in our dataset. For a full list of the metrics available in `tf.keras` check out the [metrics documentation](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/metrics#classes).

이것들이 이 과정을 통해서 설정할 주요 파라미터들이다. [TensorFlow documentation](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#compile)에서 다른 설정 파라미터들을 확인할 수 있다.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Taking a Look at the Loss and Accuracy Before Training

model을 trian시키기 전에, random weights를 사용할때 model이 어떻게 수행되는지를 살펴보자. images의 single batch를 un-trained model에 전달할때 `loss` 와 `accuracy` values을 살펴보자. 이를 위해서  `.evaluate(data, true_labels)` method를 사용한다. `.evaluate(data, true_labels)` method는 주어진 `data`에 model의 predicted output과 주어진 `true_labels`과 비교하고 `loss` 와 `accuracy` values을 반환한다.

In [ ]:
for image_batch, label_batch in training_batches.take(1):
    loss, accuracy = model.evaluate(image_batch, label_batch)

print('\nLoss before training: {:,.3f}'.format(loss))
print('Accuracy before training: {:.3%}'.format(accuracy))

## Training the Model

training set에 모든 images를 사용하여 model을 train시키자. 전체 dataset을 한번 pass시키는 것을 *epoch*라고 부른다. `.fit` method를 사용해서 epochs의 주어진 숫자에 대한 model 훈련시키며 아래와 같다:

In [ ]:
EPOCHS = 5

history = model.fit(training_batches, epochs = EPOCHS)

`.fit` method는 `History` object를 반환하며 여기에는 validation 정확도와 loss values 뿐만 아니라 이후 epochs에 대한 training accuracy 와 loss values의 기록을 포함하고 있다. 이후 과정에서 history object에 대해서 이야기해보자.

학습된 model을 가지고 prediciton을 확인할 수 있다.

In [ ]:
for image_batch, label_batch in training_batches.take(1):
    ps = model.predict(image_batch)
    first_image = image_batch.numpy().squeeze()[0]
  
  
fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
ax1.imshow(first_image, cmap = plt.cm.binary)
ax1.axis('off')
ax2.barh(np.arange(10), ps[0])
ax2.set_aspect(0.1)
ax2.set_yticks(np.arange(10))
ax2.set_yticklabels(np.arange(10))
ax2.set_title('Class Probability')
ax2.set_xlim(0, 1.1)
plt.tight_layout()

WOW!! 이제 우리 network은 훌륭하다. images에서 숫자를 정확하게 예측할 수 있다. images의 single batch에 대해서 다시 loss와 accuracy values를 살펴보자.

In [ ]:
for image_batch, label_batch in training_batches.take(1):
    loss, accuracy = model.evaluate(image_batch, label_batch)

print('\nLoss after training: {:,.3f}'.format(loss))
print('Accuracy after training: {:.3%}'.format(accuracy))

> **연습문제:** 784 input units, a hidden layer with 128 units, 다음으로 a hidden layer with 64 units, 다음으로 a hidden layer with 32 units 그리고 마지막으로 an output layer with 10 units 인 네트워크를 생성하자. 모든 hidden layers에 대해서 ReLu activation function을 그리고 output layer에 대해서는 softmax activation function을 사용하자. 다음으로 `adam` optimizer, `sparse_categorical_crossentropy` loss function, `accuracy` metric 를 사용하여 model을 컴파일한다. image의 single batch에 대한 un-trained model의 loss와 정확도를 출력한다.

In [ ]:
## Solution


print('\nLoss before training: {:,.3f}'.format(loss))
print('Accuracy before training: {:.3%}'.format(accuracy))

> **연습문제:** 위에서 생성한 model을 5 epochs 동안 train시키고 나서 images의 single batch에 대해서 trained model의 loss와 정확도를 출력한다.

In [ ]:
## Solution


print('\nLoss after training: {:,.3f}'.format(loss))
print('Accuracy after training: {:.3%}'.format(accuracy))

> **Exercise:** Plot the prediction of the model you created and trained above on a single image from the training set. Also plot the probability predicted by your model for each digit.

In [ ]:
## Solution


## Automatic Differentiation

TensorFlow가 계산하고 backpropagation을 위해 필요한 gradients를 추적하는 방법에  대해서 잠시 알아보자. TensorFlow는 자동 미분 연산을 기록하는 class를 제공한다. 그 class는 `tf.GradientTape`이다. 자동 미분은 간단히 "autodiff"로 알려져 있지만 컴퓨터에서 수치 함수의 미분을 효과적이고 정확하게 처리하기 위해 사용되는 기술중에 하나다.

`tf.GradientTape`은 "watched"되는 tensors에서 수행되는 연산을 추적해서 동작한다. 기본적으로 `tf.GradientTape`은 자동으로 어떤 훈련 가능한 변수든 "watch"하며 model에서는 weights해 해당된다. 훈련가능한 변수는 `trainable=True`를 가진다. `tf.keras`를 가지는 model을 생성할때 모든 파라미터들은 `trainable = True`로 초기화한다. tensor는 watch method를 호출해서 수동으로 "watched"될 수 있다.

간단한 예제를 보자. 다음과 같은 식을 따른다.:

$$
y = x^2
$$

`x`에 대한 `y`의 미분은 다음과 같이 주어진다 :

$$
\frac{d y}{d x} = 2x
$$

`tf.GradientTape`를 사용해서 tensor `x`에 대한 tensor `y`의 미분을 계산하자.:

In [ ]:
# Set the random seed so things are reproducible
tf.random.set_seed(7)

# Create a random tensor
x = tf.random.normal((2,2))

# Calculate gradient
with tf.GradientTape() as g:
    g.watch(x)
    y = x ** 2
    
dy_dx = g.gradient(y, x)

# Calculate the actual gradient of y = x^2
true_grad = 2 * x

# Print the gradient calculated by tf.GradientTape
print('Gradient calculated by tf.GradientTape:\n', dy_dx)

# Print the actual gradient of y = x^2
print('\nTrue Gradient:\n', true_grad)

# Print the maximum difference between true and calculated gradient
print('\nMaximum Difference:', np.abs(true_grad - dy_dx).max())

`tf.GradientTape` class는 이런 연산을 유지하고 각각에 대해서 gradient를 계산하는 방법을 알고 있다. 이런 방식으로 어떤 하나의 tensor에 대해서 연산의 chain으로 gradients를 계산하는 것이 가능하다.

`tf.GradientTape`와 훈련 가능한 변수에 대해서 상세한 정보는 아래 링크는 참고하자.

* [Gradient Tape](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/GradientTape)

* [TensorFlow Variables](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/Variable)

다음에는 좀더 복잡한 dataset을 neural network로 훈련시키기 위한 코드를 작성해볼 예정이다.